In [3]:
import logging
import tensorflow as tf
from struct import pack, unpack
import numpy as np
from tensorflow.keras import Model, layers
from tensorflow.keras import backend as K
import random, math
from enum import  Enum
import sys
from MLP import MLP

RandomBitPos=-1
class InjectionType(Enum):
	Weights=0
	Outputs=1
class BitFaultType(Enum):
	Zero=0
	Rand=1
	Flip=2
class LayerSelectionType(Enum):
	Single_Layer=0
	Multiple_Layer=1

def bitflip(f, pos):
	
	""" Single bit-flip in 32 bit floats """

	f_ = pack('f', f)
	b = list(unpack('BBBB', f_))
	[q, r] = divmod(pos, 8)
	b[q] ^= 1 << r
	f_ = pack('BBBB', *b)
	f = unpack('f', f_)
	return f[0]


def inject(mlp:MLP,Injection:InjectionType,FType:LayerSelectionType,BitFault:BitFaultType,FaultCount:int, log_level="ERROR"):
	# Logging setup
	logging.basicConfig()
	logging.getLogger().setLevel(log_level)
	logging.debug("Logging level set to {0}".format(log_level))
	#self.Model = mlp.Model # No more passing or using a session variable in TF v2
	# Call the corresponding FI function
	if(Injection==InjectionType.Weights):
		layer_states(mlp.Model,FType,BitFault,FaultCount,RandomBitPos,mlp.x_test_images)
	elif(Injection==InjectionType.Outputs):
		sdc=0.0
		res=None
		for i in range(1,len(mlp.x_test_images)):
			res=layer_outputs(mlp.Model,FType,BitFault,FaultCount,RandomBitPos,mlp.x_test_images[i:i+1])
			if(i%200==0):
				ShowPercentage((float(i)/len(mlp.y_test_labels))*100)
			if(res != mlp.y_test_labels[i:i+1]):
				sdc = sdc + 1.
		ShowPercentage(100)
		test_acc=1.0-sdc/len(mlp.y_test_labels)
		#print("Accuracy after faults:", test_acc)
		return test_acc

def layer_states(model, FType:LayerSelectionType,BitFault:BitFaultType,FaultCount:int,FaultyBitPos:int=RandomBitPos, Dataset=None):
		
	""" FI in layer states """
		
	if(FType == LayerSelectionType.Single_Layer):

		""" Single layer fault injection mode """

		logging.info("Starting fault injection in a random layer")

		# Retrieve type and amount of fault
			
		fiSz = FaultCount

		# Choose a random layer for injection
		randnum = random.randint(0, len(model.trainable_variables) - 1)

		# Get layer states info
		v = model.trainable_variables[randnum]
		num = v.shape.num_elements()

		if(BitFault == BitFaultType.Zero):
			fiSz = (fiSz * num) / 100
			fiSz = math.floor(fiSz)

		# Choose the indices for FI
		ind = random.sample(range(num), fiSz)

		# Unstack elements into a single dimension
		elem_shape = v.shape
		v_ = tf.identity(v)
		v_ = tf.keras.backend.flatten(v_)
		v_ = tf.unstack(v_)

		# Inject the specified fault into the randomly chosen values
		if(BitFault == BitFaultType.Zero):
			for item in ind:
				v_[item] = 0.
		elif(BitFault == BitFaultType.Rand):
			for item in ind:
				v_[item] = np.random.random()
		elif(BitFault == BitFaultType):
			for item in ind:
				val = v_[item]
					
				# If random bit chosen to be flipped
				if(FaultyBitPos == RandomBitPos):
					pos = random.randint(0, 31)

				# If bit position specified for flip
				else:
					pos = int(FaultyBitPos)
				val_ = bitflip(val, pos)
				v_[item] = val_

		# Reshape into original dimensions and store the faulty tensor
		v_ = tf.stack(v_)
		v_ = tf.reshape(v_, elem_shape)
		v.assign(v_)

		logging.info("Completed injections... exiting")

	elif(FType == LayerSelectionType.Multiple_Layer):

		""" Multiple layer fault injection mode """

		logging.info("Starting fault injection in all layers")

		# Retrieve type and amount of fault
			
		fiSz = FaultCount

		# Loop through each available layer in the model
		for n in range(len(model.trainable_variables) - 1):

			# Get layer states info
			v = model.trainable_variables[n]
			num = v.shape.num_elements()

			if(BitFault == BitFaultType.Zero):
				fiSz = (fiSz * num) / 100
				fiSz = math.floor(fiSz)

			# Choose the indices for FI
			ind = random.sample(range(num), fiSz)

			# Unstack elements into a single dimension
			elem_shape = v.shape
			v_ = tf.identity(v)
			v_ = tf.keras.backend.flatten(v_)
			v_ = tf.unstack(v_)

			# Inject the specified fault into the randomly chosen values
			if(BitFault == BitFaultType.Zero):
				for item in ind:
					v_[item] = 0.
			elif(BitFault == BitFaultType.Rand):
				for item in ind:
					v_[item] = np.random.random()
			elif(BitFault == BitFaultType.Flip):
				for item in ind:
					val = v_[item]

					# If random bit chosen to be flipped
					if(FaultyBitPos == RandomBitPos):
						pos = random.randint(0, 31)

					# If bit position specified for flip
					else:
						pos = int(FaultyBitPos)
					val_ = bitflip(val, pos)
					v_[item] = val_

			# Reshape into original dimensions and store the faulty tensor
			v_ = tf.stack(v_)
			v_ = tf.reshape(v_, elem_shape)
			v.assign(v_)

		logging.info("Completed injections... exiting")


def layer_outputs(model,FType:LayerSelectionType,BitFault:BitFaultType,FaultCount:int,FaultyBitPos:int=RandomBitPos, Dataset_Sample=None):

	""" FI in layer computations/outputs """

	if(FType == LayerSelectionType.Single_Layer):

		""" Single layer fault injection mode """

		logging.info("Starting fault injection in a random layer")

		# Retrieve type and amount of fault
			
		fiSz = FaultCount

		# Get the input for which dynamic injection is to be done
		x_test = Dataset_Sample

		# Choose a random layer for injection
		randnum = random.randint(0, len(model.layers) - 2)

		fiLayer = model.layers[randnum]

		# Get the outputs of the chosen layer
		get_output = K.function([model.layers[0].input], [fiLayer.output])
		fiLayerOutputs = get_output([x_test])

		# Unstack elements into a single dimension
		elem_shape = fiLayerOutputs[0].shape
		fiLayerOutputs[0] = fiLayerOutputs[0].flatten()
		num = fiLayerOutputs[0].shape[0]

		if(BitFault == BitFaultType.Zero):
			fiSz = (fiSz * num) / 100
			fiSz = math.floor(fiSz)

		# Choose the indices for FI
		ind = random.sample(range(num), fiSz)

		# Inject the specified fault into the randomly chosen values
		if(BitFault == BitFaultType.Zero):
			for item in ind:
				fiLayerOutputs[0][item] = 0.
		elif(BitFault == BitFaultType.Rand):
			for item in ind:
				fiLayerOutputs[0][item] = np.random.random()
		elif(BitFault == BitFaultType.Flip):
			for item in ind:
				val = fiLayerOutputs[0][item]
				if(FaultyBitPos == RandomBitPos):
					pos = random.randint(0, 31)
				else:
					pos = int(FaultyBitPos)
				val_ = bitflip(val, pos)
				fiLayerOutputs[0][item] = val_

		# Reshape into original dimensions and get the final prediction
		fiLayerOutputs[0] = fiLayerOutputs[0].reshape(elem_shape)
		get_pred = K.function([model.layers[randnum + 1].input], [model.layers[-1].output])
		pred = get_pred([fiLayerOutputs])

		# Uncomment below line and comment next two lines for ImageNet models
		# return pred
		labels = np.argmax(pred, axis=-1)
		return labels[0]
			
		logging.info("Completed injections... exiting")

	elif(FType == LayerSelectionType.Multiple_Layer):

		""" Multiple layer fault injection mode """

		logging.info("Starting fault injection in all layers")

		# Retrieve type and amount of fault
			
		fiSz = FaultCount

		# Get the input for which dynamic injection is to be done
		x_test = Dataset_Sample

		# Get the outputs of the first layer
		get_output_0 = K.function([model.layers[0].input], [model.layers[1].output])
		fiLayerOutputs = get_output_0([x_test])

		# Loop through each available layer in the model
		for n in range(1, len(model.layers) - 2):

			# Unstack elements into a single dimension
			elem_shape = fiLayerOutputs[0].shape
			fiLayerOutputs[0] = fiLayerOutputs[0].flatten()
			num = fiLayerOutputs[0].shape[0]
			if(BitFault == BitFaultType.Zero):
				fiSz = (fiSz * num) / 100
				fiSz = math.floor(fiSz)

			# Choose the indices for FI
			ind = random.sample(range(num), fiSz)

			# Inject the specified fault into the randomly chosen values
			if(BitFault == BitFaultType.Zero):
				for item in ind:
					fiLayerOutputs[0][item] = 0.
			elif(BitFault == BitFaultType.Rand):
				for item in ind:
					fiLayerOutputs[0][item] = np.random.random()
			elif(BitFault == BitFaultType.Flip):
				for item in ind:
					val = fiLayerOutputs[0][item]
					if(FaultyBitPos == RandomBitPos):
						pos = random.randint(0, 31)
					else:
						pos = int(FaultyBitPos)
					val_ = bitflip(val, pos)
					fiLayerOutputs[0][item] = val_

			# Reshape into original dimensions
			fiLayerOutputs[0] = fiLayerOutputs[0].reshape(elem_shape)

			"""
			Check if last but one layer reached;
			if not, replace fiLayerOutputs with the next prediction to continue
			"""
			if(n != (len(model.layers) - 3)):
				get_output = K.function([model.layers[n+1].input], [model.layers[n+2].output])
				fiLayerOutputs = get_output([fiLayerOutputs])

			# Get final prediction
			get_pred = K.function([model.layers[len(model.layers)-1].input], [model.layers[-1].output])
			pred = get_pred([fiLayerOutputs])

			# Uncomment below line and comment next two lines for ImageNet models
			# return pred
			labels = np.argmax(pred, axis=-1)
			return labels[0]
				
			logging.info("Completed injections... exiting")				

def ShowPercentage(percent:int):
    sys.stdout.write('\r')
    sys.stdout.write('Fault Injection : %'+str(percent))
    sys.stdout.flush()
               

In [57]:
def Weight_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork()
    model.LoadModel(r'model\model.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Multiple_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()
 
        
Weight_Fault_Injection()

Creating Network...
dataset is ready
results for  10 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 30ms/step - loss: 0.0830 - accuracy: 0.9769
results for  11 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 32ms/step - loss: 0.0839 - accuracy: 0.9766
results for  12 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 32ms/step - loss: 0.0866 - accuracy: 0.9757
results for  13 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 33ms/step - loss: 0.0918 - accuracy: 0.9747
results for  14 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 33ms/step - loss: 0.0931 - accuracy: 0.9746
results for  15 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 6s 20ms/step - loss: 0.0912 - accuracy: 0.9747


In [ ]:
def Weight_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork()
    model.LoadModel(r'model\model.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()
 
        
Weight_Fault_Injection()

In [58]:
def Output_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork()
    model.LoadModel(r'model\model.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Outputs, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()

 
        
Output_Fault_Injection()

Creating Network...
dataset is ready
Fault Injection : %100099999999999994results for  10 fault in weights and single layer
Fault Injection : %100099999999999994results for  11 fault in weights and single layer
Fault Injection : %100099999999999994results for  12 fault in weights and single layer
Fault Injection : %100099999999999994results for  13 fault in weights and single layer
Fault Injection : %100099999999999994results for  14 fault in weights and single layer
Fault Injection : %100099999999999994results for  15 fault in weights and single layer


In [ ]:
def Output_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork()
    model.LoadModel(r'model\model.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Multiple_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()
 
        
Output_Fault_Injection()

In [60]:
def Output_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork()
    model.LoadModel(r'model\model.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Outputs, FType=LayerSelectionType.Multiple_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()
 
        
Output_Fault_Injection()

Creating Network...
dataset is ready
Fault Injection : %100099999999999994results for  10 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 30ms/step - loss: 0.0835 - accuracy: 0.9768
Fault Injection : %100099999999999994results for  11 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 8s 27ms/step - loss: 0.0835 - accuracy: 0.9768
Fault Injection : %100099999999999994results for  12 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 9s 29ms/step - loss: 0.0835 - accuracy: 0.9768
Fault Injection : %100099999999999994results for  13 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 13s 40ms/step - loss: 0.0835 - accuracy: 0.9768
Fault Injection : %100099999999999994results for  14 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 11s 35ms/step - loss: 0.0835 - accuracy: 0.

In [ ]:
def Output_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork()
    model.LoadModel(r'model\model.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Outputs, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        
Output_Fault_Injection()

In [4]:
model=MLP()
model.PrepareDataset()
model.CreateNetwork_With_Dropout()
model.Train(10)
model.Test()
model.SaveModel(r'model\model_withdropout.h5')

dataset is ready
Creating Network With Dropout started...
Training Network started at 2022-05-18 22:19:08.080407....
Epoch 1/10
469/469 [==============================] - 68s 138ms/step - loss: 0.2412 - accuracy: 0.9250 - val_loss: 0.1296 - val_accuracy: 0.9604
Epoch 2/10
469/469 [==============================] - 69s 147ms/step - loss: 0.1018 - accuracy: 0.9696 - val_loss: 0.0821 - val_accuracy: 0.9762
Epoch 3/10
469/469 [==============================] - 69s 147ms/step - loss: 0.0784 - accuracy: 0.9754 - val_loss: 0.0693 - val_accuracy: 0.9788
Epoch 4/10
469/469 [==============================] - 71s 151ms/step - loss: 0.0616 - accuracy: 0.9809 - val_loss: 0.0714 - val_accuracy: 0.9790
Epoch 5/10
469/469 [==============================] - 72s 154ms/step - loss: 0.0535 - accuracy: 0.9833 - val_loss: 0.0636 - val_accuracy: 0.9816
Epoch 6/10
469/469 [==============================] - 70s 150ms/step - loss: 0.0437 - accuracy: 0.9862 - val_loss: 0.0715 - val_accuracy: 0.9807
Epoch 7/10
46

In [9]:
def Weight_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork_With_Dropout()
    model.LoadModel(r'model\model_withdropout.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Zero, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()

    # for i in [10,11,12,13,14,15]:

    #     # injecting weight fault in single layer
    #     inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Multiple_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
    #     print("results for ",i ,"fault in weights and single layer")
    #     model.Test()
 
        
Weight_Fault_Injection()

Creating Network With Dropout started...
dataset is ready
results for  10 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 11s 34ms/step - loss: 0.0710 - accuracy: 0.9818
results for  11 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 11s 34ms/step - loss: 0.0708 - accuracy: 0.9817
results for  12 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 30ms/step - loss: 0.0670 - accuracy: 0.9816
results for  13 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 31ms/step - loss: 0.0641 - accuracy: 0.9815
results for  14 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 32ms/step - loss: 0.0642 - accuracy: 0.9812
results for  15 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 32ms/step - loss: 0.0645 - accuracy: 0.9812


In [13]:
def Weight_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork_With_Dropout()
    model.LoadModel(r'model\model_withdropout.h5')

    # loading dataset
    model.PrepareDataset()   

    # for i in [10,11,12,13,14,15]:

    #     # injecting weight fault in single layer
    #     inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
    #     print("results for ",i ,"fault in weights and single layer")
    #     model.Test()

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Weights, FType=LayerSelectionType.Multiple_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")
        model.Test()
 
        
Weight_Fault_Injection()

Creating Network With Dropout started...
dataset is ready
results for  10 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 30ms/step - loss: 0.0825 - accuracy: 0.9795
results for  11 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 7s 21ms/step - loss: 0.0847 - accuracy: 0.9794
results for  12 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 8s 25ms/step - loss: 0.0852 - accuracy: 0.9799
results for  13 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 30ms/step - loss: 0.0902 - accuracy: 0.9790
results for  14 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 10s 31ms/step - loss: 0.0945 - accuracy: 0.9782
results for  15 fault in weights and single layer
evaluating ... 
313/313 [==============================] - 6s 20ms/step - loss: 0.0981 - accuracy: 0.9779


In [14]:
def Output_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork_With_Dropout()
    model.LoadModel(r'model\model_withdropout.h5')

    # loading dataset
    model.PrepareDataset()   

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Outputs, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in weights and single layer")

        
Output_Fault_Injection()

Creating Network With Dropout started...
dataset is ready
Fault Injection : %100099999999999994results for  10 fault in weights and single layer
Fault Injection : %100099999999999994results for  11 fault in weights and single layer
Fault Injection : %100099999999999994results for  12 fault in weights and single layer
Fault Injection : %100099999999999994results for  13 fault in weights and single layer
Fault Injection : %100099999999999994results for  14 fault in weights and single layer
Fault Injection : %100099999999999994results for  15 fault in weights and single layer


In [ ]:
def Output_Fault_Injection():
    # loading the network
    model=MLP()
    model.CreateNetwork_With_Dropout()
    model.LoadModel(r'model\model_withdropout.h5')

    # loading dataset
    model.PrepareDataset()   

    # for i in [10,11,12,13,14,15]:

    #     # injecting weight fault in single layer
    #     inject(mlp=model, Injection=InjectionType.Outputs, FType=LayerSelectionType.Single_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
    #     print("results for ",i ,"fault in weights and single layer")

    for i in [10,11,12,13,14,15]:

        # injecting weight fault in single layer
        inject(mlp=model, Injection=InjectionType.Outputs, FType=LayerSelectionType.Multiple_Layer, BitFault=BitFaultType.Rand, FaultCount=i)
        print("results for ",i ,"fault in output and multi layer") 
        
Output_Fault_Injection()